## pickleから辞書を取得

In [1]:
# # （tsv形式の辞書からpickleを取得）

# with open('data_extract/ontology.tsv', mode='r', newline='', encoding='utf-8') as f:
#     tsv_reader = csv.reader(f, delimiter='\t')
#     data = [row for row in tsv_reader if row != []]

# new_data = []
# key = data[0]

# for d in data[1:]:
#     data_d = {}
#     for i in range(len(key)):
#         data_d[key[i]] = d[i]
#     new_data.append(data_d)

# import pickle
# with open("data_extract/ontology.pickle", 'wb') as f:
#     pickle.dump(new_data,f)

import pickle
with open("data_extract/ontology.pickle", 'rb') as f:
    data = pickle.load(f)

## DeepL API準備

In [57]:
with open('data_secret/DeepL_api.txt', 'r', encoding='UTF-8') as f:
    k = f.read()

In [58]:
import requests

def ENtoJA(text):
    # URLクエリに仕込むパラメータの辞書を作っておく
    params = {"auth_key": k,
                "text": text,
                "source_lang": 'EN', # 入力テキストの言語を英語に設定
                "target_lang": 'JA'  # 出力テキストの言語を日本語に設定（JPではなくJAなので注意）
             }

    # パラメータと一緒にPOSTする
    request = requests.post("https://api-free.deepl.com/v2/translate", data=params) # free用のURL、有料版はURLが異なります
    result = request.json()
    return result["translations"][0]["text"]

In [65]:
for d in data:
    if "trans_name" in d:
        pass
    else:
        d["trans_name"] = ENtoJA(d["en_name"])
        print('\r%s                                            ' %(d["trans_name"]), end='')

In [72]:
import pickle
with open("DeepL_11057in50640.bin", 'wb') as f:
    pickle.dump(data,f)
    
# 翻訳結果
# ・50640項目中11057の地点でAPIの利用制限到達，翻訳精度微妙
# ・英語を直に日本語読みとかのほうがまだいいかも
# ・作成したデータは上記のバイナリファイルに保存してtrushに移動済み

## ローマ字読み付与

In [76]:
# !pip install pykakasi

In [77]:
import pykakasi

kakasi = pykakasi.kakasi() # インスタンスの作成
kakasi.setMode('H', 'a') # ひらがなをローマ字に変換するように設定
kakasi.setMode('K', 'a') # カタカナをローマ字に変換するように設定
kakasi.setMode('J', 'a') # 漢字をローマ字に変換するように設定
conversion = kakasi.getConverter() # 上記モード設定の適用


for d in data:
    if "romaji_name" in d:
        pass
    else:
        d["romaji_name"] = conversion.do(d["ja_name"])
        print('\r%s                                            ' %(d["romaji_name"]), end='')

<ipython-input-77-199bc52e4aae>:4: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('H', 'a') # ひらがなをローマ字に変換するように設定
<ipython-input-77-199bc52e4aae>:5: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('K', 'a') # カタカナをローマ字に変換するように設定
<ipython-input-77-199bc52e4aae>:6: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  kakasi.setMode('J', 'a') # 漢字をローマ字に変換するように設定
<ipython-input-77-199bc52e4aae>:7: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  conversion = kakasi.getConverter() # 上記モード設定の適用
<ipython-input-77-199bc52e4aae>:14: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
  d["romaji_na

oodarumainko                                                                           

## ネットワーク作成　（&親子tsv作成)

In [137]:
!pip install networkx

You should consider upgrading via the 'c:\users\teramoto\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
child_parent_data = []
for d in data:
    child_parent_data.append({"id":d["id"],"parent_taxon":d["parent_taxon"]})

import csv
with open('data_extract/child_parent.tsv','w',encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = ["id","parent_taxon"],delimiter = "\t")
    writer.writeheader()
    writer.writerows(child_parent_data)

In [4]:
import networkx as nx
G = nx.DiGraph() #無向グラフを作成 

In [5]:
for d in child_parent_data:
    G.add_edge(d["parent_taxon"],d["id"])

In [6]:
from matplotlib import pyplot as plt

# # pos = nx.spring_layout(G, seed=0)
# plt.figure() #グラフエリアのサイズ
# nx.draw_networkx(G) #グラフの描画(おまかせ)
# plt.show() #グラフの描画

In [9]:
!pip install --upgrade matplotlib

ERROR: Could not install packages due to an EnvironmentError: [WinError 5] アクセスが拒否されました。: 'C:\\Users\\Teramoto\\AppData\\Local\\Programs\\Python\\Python39\\Lib\\site-packages\\~atplotlib\\backends\\_backend_agg.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'c:\users\teramoto\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.4.2
    Uninstalling matplotlib-3.4.2:
      Successfully uninstalled matplotlib-3.4.2


In [ ]:
from matplotlib import pyplot as plt

print(1)
plt.figure(figsize=(108000,108000))
print(2)
pos = nx.spring_layout(G)
print(3)
nx.draw_networkx(G,pos)

print(4)
plt.axis("off")
print(5)
plt.savefig("default.png")
print(6)
plt.show()

1
2
